In [ ]:
import csv
import math
import random
import tkinter as tk
from tkinter import ttk, messagebox, filedialog

# Data model

class Route:
    def __init__(self, route_name, crime_rate, lighting, cctv, crowd, safety=None, person_name=None):
        self.route_name = route_name
        self.person_name = person_name or ""
        # numeric fields (store as floats)
        try:
            self.crime_rate = float(crime_rate)
        except Exception:
            self.crime_rate = 5.0
        try:
            self.lighting = float(lighting)
        except Exception:
            self.lighting = 5.0
        try:
            self.cctv = float(cctv)
        except Exception:
            self.cctv = 5.0
        try:
            self.crowd = float(crowd)
        except Exception:
            self.crowd = 5.0
        try:
            self.safety = float(safety) if safety is not None and safety != "" else None
        except Exception:
            self.safety = None

    def features(self):
        return [1.0, 10.0 - self.crime_rate, self.lighting, self.cctv, self.crowd]

    def baseline_score(self):
        return (
            (10.0 - self.crime_rate) * 2.0 +
            self.lighting * 1.5 +
            self.cctv * 3.0 +
            self.crowd * 1.0
        )

# Tiny linear model (from-scratch)
class TinyLinearModel:
    def __init__(self, n_features):
        self.n_features = n_features
        self.w = [random.uniform(-0.1, 0.1) for _ in range(n_features)]

    def predict(self, x):
        return sum(wi * xi for wi, xi in zip(self.w, x))

    def mse(self, X, y):
        n = len(X)
        if n == 0:
            return 0.0
        return sum((self.predict(xi) - yi) ** 2 for xi, yi in zip(X, y)) / n

    def train(self, X, y, lr=0.001, epochs=1000, verbose=False):
        n = len(X)
        if n == 0:
            return
        m = self.n_features
        for ep in range(epochs):
            grad = [0.0] * m
            for xi, yi in zip(X, y):
                pred = self.predict(xi)
                err = pred - yi
                for j in range(m):
                    grad[j] += (2.0 / n) * err * xi[j]
            for j in range(m):
                self.w[j] -= lr * grad[j]
            if verbose and (ep % max(1, (epochs // 5)) == 0):
                print(f"Epoch {ep} MSE={self.mse(X, y):.4f}")

# Analyzer

class SafeRouteAnalyzer:
    def __init__(self):
        self.routes = {}  
        self.model = None
        self.trained = False
        self._norm = None

    def load_csv(self, path):
        self.routes = {}
        with open(path, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            if reader.fieldnames is None:
                raise ValueError("CSV has no header row")

            header_map = {h.strip().lower().replace(" ", "_"): h for h in reader.fieldnames}

            route_keys = ["route", "route_name", "routename", "route id", "name", "route_name_field", "route_name_field"]
            person_keys = ["person_name", "person", "driver", "name", "fullname"]
            numeric_keys = {
                "crime_rate": ["crime_rate", "crime", "crime_rate_value"],
                "lighting": ["lighting", "light"],
                "cctv": ["cctv", "cctv_count", "camera"],
                "crowd": ["crowd", "crowd_density", "density"],
                "safety": ["safety", "safety_score", "score"]
            }

            def find_col(possible_list):
                for k in possible_list:
                    if k in header_map:
                        return header_map[k]
                return None

            route_col = find_col(route_keys)
            if not route_col:
                raise KeyError(f"CSV missing route column. Expected one of: {route_keys}")

            person_col = find_col(person_keys)

            found = {}
            for logical, cand in numeric_keys.items():
                col = find_col(cand)
                found[logical] = col  
                
            core_missing = [k for k in ("crime_rate", "lighting", "cctv", "crowd") if not found[k]]
            if core_missing:
                raise KeyError(f"CSV missing required numeric columns: {core_missing}")

            row_num = 0
            for row in reader:
                row_num += 1
          
                def g(col):
                    return row.get(col, "").strip() if col else ""

                route_name = g(route_col) or f"Route_{row_num}"
                person_name = g(person_col) if person_col else ""
                crime_rate = g(found["crime_rate"])
                lighting = g(found["lighting"])
                cctv = g(found["cctv"])
                crowd = g(found["crowd"])
                safety = g(found["safety"]) if found["safety"] else ""

                r = Route(route_name, crime_rate or "5", lighting or "5", cctv or "5", crowd or "5", safety if safety != "" else None, person_name)
                self.routes[r.route_name] = r

    def available_routes(self):
        return list(self.routes.keys())

    def baseline_compare(self, r1name, r2name):
        r1 = self.routes.get(r1name)
        r2 = self.routes.get(r2name)
        if not r1 or not r2:
            raise ValueError("Route names not found")
        return r1.baseline_score(), r2.baseline_score()

    def prepare_training_data(self):
        X = []
        y = []
        for r in self.routes.values():
            feats = r.features()
            X.append(feats)

            y.append(r.safety if r.safety is not None else r.baseline_score())
        return X, y

    def train_model(self, lr=0.0005, epochs=2000):
        X, y = self.prepare_training_data()
        if not X:
            raise RuntimeError("No data to train on")

        m = len(X[0])
        n = len(X)

        means = [sum(X[i][j] for i in range(n)) / n for j in range(m)]
        stds = []
        for j in range(m):
            var = sum((X[i][j] - means[j]) ** 2 for i in range(n)) / n
            std = math.sqrt(var) if var > 1e-12 else 1.0
            stds.append(std)

        Xn = [[(X[i][j] - means[j]) / stds[j] for j in range(m)] for i in range(n)]
        # build and train model
        self.model = TinyLinearModel(m)
        self.model.train(Xn, y, lr=lr, epochs=epochs)
        self.trained = True
        self._norm = (means, stds)

    def model_score(self, route_name):
        if not self.trained:
            raise RuntimeError("Model not trained yet")
        r = self.routes.get(route_name)
        if not r:
            raise ValueError("Route not found")
        feats = r.features()
        means, stds = self._norm
        fn = [(feats[j] - means[j]) / (stds[j] if stds[j] != 0 else 1.0) for j in range(len(feats))]
        return self.model.predict(fn)
    
# GPS Simulation utilities

def interpolate_points(lat1, lon1, lat2, lon2, n=30):
    pts = []
    for i in range(n + 1):
        t = i / n
        lat = lat1 + (lat2 - lat1) * t
        lon = lon1 + (lon2 - lon1) * t
        pts.append((lat, lon))
    return pts

def simulate_route_safety(pts):
    vals = []
    for (lat, lon) in pts:
        base = 50 + (math.sin(lat * 3.3) + math.cos(lon * 2.1)) * 10
        noise = random.uniform(-8, 8)
        vals.append(max(0, min(100, base + noise)))
    return vals

# GUI

class SafeRouteGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("SafeRoute — Upgraded")
        self.analyzer = SafeRouteAnalyzer()
        self._build_ui()

    def _build_ui(self):
        frm = ttk.Frame(self.root, padding=8)
        frm.pack(fill="both", expand=True)

        top = ttk.Frame(frm)
        top.pack(fill="x", pady=4)

        ttk.Label(top, text="Load CSV:").pack(side="left")
        self.load_btn = ttk.Button(top, text="Open CSV", command=self._on_open_csv)
        self.load_btn.pack(side="left", padx=4)

        ttk.Label(top, text="Routes:").pack(side="left", padx=(20, 4))
        self.route_combo = ttk.Combobox(top, values=[], state="readonly", width=28)
        self.route_combo.pack(side="left")
        self.route_combo2 = ttk.Combobox(top, values=[], state="readonly", width=28)
        self.route_combo2.pack(side="left", padx=4)

        self.compare_btn = ttk.Button(top, text="Compare (baseline)", command=self._on_compare)
        self.compare_btn.pack(side="left", padx=4)
        self.train_btn = ttk.Button(top, text="Train AI scorer", command=self._on_train)
        self.train_btn.pack(side="left", padx=6)
        self.compare_model_btn = ttk.Button(top, text="Compare (AI model)", command=self._on_compare_model)
        self.compare_model_btn.pack(side="left", padx=4)

        coord_fr = ttk.LabelFrame(frm, text="GPS Route Simulation")
        coord_fr.pack(fill="x", pady=8)

        c1 = ttk.Frame(coord_fr)
        c1.pack(fill="x", pady=2)
        ttk.Label(c1, text="Start lat,lon:").pack(side="left")
        self.start_entry = ttk.Entry(c1, width=20)
        self.start_entry.pack(side="left", padx=4)
        self.start_entry.insert(0, "12.95,77.59")  # sample coords

        ttk.Label(c1, text="End lat,lon:").pack(side="left", padx=8)
        self.end_entry = ttk.Entry(c1, width=20)
        self.end_entry.pack(side="left", padx=4)
        self.end_entry.insert(0, "12.97,77.60")

        self.simulate_btn = ttk.Button(coord_fr, text="Simulate Route & Heatmap", command=self._on_simulate)
        self.simulate_btn.pack(pady=4)

        self.canvas = tk.Canvas(frm, width=820, height=420, bg="white")
        self.canvas.pack(pady=6)

        self.status = ttk.Label(self.root, text="Ready", relief="sunken", anchor="w")
        self.status.pack(side="bottom", fill="x")

    def _set_status(self, txt):
        self.status.config(text=txt)
        self.root.update_idletasks()

    def _on_open_csv(self):
        path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if not path:
            return
        try:
            self._set_status("Loading CSV...")
            self.analyzer.load_csv(path)
            routes = self.analyzer.available_routes()
            self.route_combo['values'] = routes
            self.route_combo2['values'] = routes
            if routes:
                self.route_combo.current(0)
                if len(routes) > 1:
                    self.route_combo2.current(1)
            self._set_status(f"Loaded {len(routes)} routes from CSV")
            messagebox.showinfo("CSV Loaded", f"Loaded {len(routes)} routes.")
        except Exception as e:
            messagebox.showerror("CSV Load Error", f"Failed to load CSV:\n{e}")
            self._set_status("Load failed")

    def _on_compare(self):
        r1 = self.route_combo.get()
        r2 = self.route_combo2.get()
        if not r1 or not r2:
            messagebox.showwarning("Selection", "Select two routes to compare.")
            return
        try:
            s1, s2 = self.analyzer.baseline_compare(r1, r2)
            out = f"{r1} baseline score: {s1:.2f}\n{r2} baseline score: {s2:.2f}\n"
            out += f"Recommended: {r1 if s1 > s2 else r2 if s2 > s1 else 'Both equal'}"
            messagebox.showinfo("Baseline Comparison", out)
            self._set_status("Baseline compared")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def _on_train(self):
        try:
            self._set_status("Training model...")
            self.analyzer.train_model(lr=0.0005, epochs=2500)
            self._set_status("Model trained (AI scorer ready)")
            messagebox.showinfo("Training complete", "AI scorer trained on CSV data.")
        except Exception as e:
            messagebox.showerror("Training Error", str(e))
            self._set_status("Training failed")

    def _on_compare_model(self):
        if not self.analyzer.trained:
            messagebox.showwarning("Model not trained", "Train the AI scorer first.")
            return
        r1 = self.route_combo.get()
        r2 = self.route_combo2.get()
        if not r1 or not r2:
            messagebox.showwarning("Selection", "Select two routes to compare.")
            return
        try:
            s1 = self.analyzer.model_score(r1)
            s2 = self.analyzer.model_score(r2)
            out = f"{r1} AI score: {s1:.2f}\n{r2} AI score: {s2:.2f}\n"
            out += f"Recommended: {r1 if s1 > s2 else r2 if s2 > s1 else 'Both equal'}"
            messagebox.showinfo("AI Comparison", out)
            self._set_status("AI model compared")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def _on_simulate(self):
        try:
            lat1, lon1 = map(float, self.start_entry.get().strip().split(","))
            lat2, lon2 = map(float, self.end_entry.get().strip().split(","))
        except Exception:
            messagebox.showerror("Input error", "Enter coords as: lat,lon (e.g. 12.95,77.59)")
            return
        pts = interpolate_points(lat1, lon1, lat2, lon2, n=60)
        vals = simulate_route_safety(pts)
        self._draw_heatmap(pts, vals)

    def _draw_heatmap(self, pts, vals):
        if not pts: 
            return
        w = int(self.canvas['width'])
        h = int(self.canvas['height'])
        lats = [p[0] for p in pts]
        lons = [p[1] for p in pts]
        minlat, maxlat = min(lats), max(lats)
        minlon, maxlon = min(lons), max(lons)
        
        pad_lat = max(1e-6, (maxlat - minlat) * 0.05) or 0.0001
        pad_lon = max(1e-6, (maxlon - minlon) * 0.05) or 0.0001
        minlat -= pad_lat; maxlat += pad_lat; minlon -= pad_lon; maxlon += pad_lon

        self.canvas.delete("all")
        prev = None
        for (lat, lon), val in zip(pts, vals):
            x = int((lon - minlon) / (maxlon - minlon) * (w - 40)) + 20
            y = int((maxlat - lat) / (maxlat - minlat) * (h - 40)) + 20
            r = int(max(0, min(255, 255 - (val / 100.0) * 255)))
            g = int(max(0, min(255, (val / 100.0) * 255)))
            b = 40
            color = f"#{r:02x}{g:02x}{b:02x}"
            self.canvas.create_oval(x - 6, y - 6, x + 6, y + 6, fill=color, outline="")
            if prev:
                self.canvas.create_line(prev[0], prev[1], x, y, fill=color, width=4)
            prev = (x, y)

        lx, ly = 10, 10
        self.canvas.create_rectangle(lx, ly, lx + 180, ly + 90, fill="#ffffff", outline="#000000")
        self.canvas.create_text(lx + 90, ly + 12, text="Heatmap legend", anchor="n", font=("Arial", 9, "bold"))
        self.canvas.create_rectangle(lx + 12, ly + 30, lx + 32, ly + 50, fill="#ff0000", outline="")
        self.canvas.create_text(lx + 40, ly + 40, text="Low safety", anchor="w", font=("Arial", 8))
        self.canvas.create_rectangle(lx + 12, ly + 55, lx + 32, ly + 75, fill="#00ff00", outline="")
        self.canvas.create_text(lx + 40, ly + 65, text="High safety", anchor="w", font=("Arial", 8))

        self._set_status("Heatmap drawn (simulated)")

# Run application
def main():
    root = tk.Tk()
    app = SafeRouteGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()